### Data Cleaning Practice  

Goals
- Practice data cleaning with Python
- Seek concise, general code
- Clearly document decisions
- Be thorough 

Datasets:
- Top grossing tours data scraped from wikipedia
    - Small dataset
- Something else
    - Large dataset
    - Seek computationally efficient ways to clean the data
        - Dask?

In [212]:
### Packages
import numpy as np
import pandas as pd
import os
from janitor import clean_names
os.chdir('C:/Users/WulfN/')

# Note: For very large datasets, spark or dask should be used in order to partition
# the computation to different clusters and dramatically reduce the processing time

### Read in Data

# this dataset is too easy, but oh well!
messy = (clean_names(
    pd.read_csv('./datasets/unclean_data_practice/wikipedia_scraped.csv'))
    .rename(columns={'adjusted_gross_in_2022_dollars_': 'adjusted_gross_in_2022_dollars', 
                     'year_s_': 'year_range'}))


In [213]:
messy

,rank,peak,all_time_peak,actual_gross,adjusted_gross_in_2022_dollars,artist,tour_title,year_range,shows,average_gross,ref_
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1]
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6]
3,4,2[7],10[7],"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7]
4,5,2[4],NaN,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]
5,6,2[4],10[9],"$305,158,363","$388,978,496",Madonna,The MDNA Tour,2012,88,"$3,467,709",[9]
6,7,2[10],NaN,"$280,000,000","$381,932,682",Celine Dion,Taking Chances World Tour,2008–2009,131,"$2,137,405",[11]
7,7,NaN,NaN,"$257,600,000","$257,600,000",Pink,Summer Carnival †,2023–2024,41,"$6,282,927",[12]
8,9,NaN,NaN,"$256,084,556","$312,258,401",Beyoncé,The Formation World Tour,2016,49,"$5,226,215",[13]
9,10,NaN,NaN,"$250,400,000","$309,141,878",Taylor Swift,The 1989 World Tour,2015,85,"$2,945,882",[14]


The wikipedia page the data is scraped from is not specificied, though it appears to be top grossing tours. 

Here are questions and areas for improvements:
- Make column names lower case and seperate words by an '_'
- 2 rows are ranked 7, why?
- remove notation like [1]
- The gross columns of ',' and '$' that need to be removed
- Tour title has unnecessary symbols (only keep letters, numbers, and spaces)
- Years could be seperated into start_year and end_year
- data types of aforementioned variables needs to be changed from 'object' to integer in most cases
- perhaps an artist level table could be made showing stats by artist

In [223]:
# Data cleaning

# Split messy strings and clean integers
messy_int = messy.select_dtypes(include='int')
messy_strings = messy.select_dtypes(include='object')

# Fix rank value in messy_int
messy_int.loc[7, 'rank'] = 8

# Clean messy strings
messy_strings = (messy_strings
          # correct column names 
                 .rename(columns={'adjusted_gross_in_2022_dollars_': 'adjusted_gross_in_2022_dollars', 
                                  'year_s_': 'year_range'})
          # split year_range into 'year_start' and 'year_end'
                 .assign(year_start = lambda x: x['year_range'].str[:4],
                         # *** Assuming year_end = year_start if no year_end provided ***
                         year_end = lambda x: x['year_range'].str[-4:]) 
                 .drop(columns=['ref_', 'year_range'])
          # remove notation like [1]
                 .replace(to_replace=r'\[.*', value='', regex = True)
          # remove ALL non number or letter symbols (aside from é)
                 .replace(to_replace=r'[^a-zA-Z0-9 é]', value='', regex=True) # r'[^a-zA-Z0-9]'  # need to include space  
)
messy_strings

,peak,all_time_peak,actual_gross,adjusted_gross_in_2022_dollars,artist,tour_title,average_gross,year_start,year_end
0,1,2,780000000,780000000,Taylor Swift,The Eras Tour,13928571,2023,2024
1,1,7,579800000,579800000,Beyoncé,Renaissance World Tour,10353571,2023,2023
2,1,2,411000000,560622615,Madonna,Sticky Sweet Tour,4835294,2008,2009
3,2,10,397300000,454751555,Pink,Beautiful Trauma World Tour,2546795,2018,2019
4,2,NaN,345675146,402844849,Taylor Swift,Reputation Stadium Tour,6522173,2018,2018
5,2,10,305158363,388978496,Madonna,The MDNA Tour,3467709,2012,2012
6,2,NaN,280000000,381932682,Celine Dion,Taking Chances World Tour,2137405,2008,2009
7,NaN,NaN,257600000,257600000,Pink,Summer Carnival,6282927,2023,2024
8,NaN,NaN,256084556,312258401,Beyoncé,The Formation World Tour,5226215,2016,2016
9,NaN,NaN,250400000,309141878,Taylor Swift,The 1989 World Tour,2945882,2015,2015


In [224]:
### Preserve column order, remove ref_ column, and include updated columns
column_order = list(messy.columns)
column_order.remove('ref_') 

column_loc = column_order.index('year_range')
column_order[column_loc:column_loc + 1] = ['year_start', 'year_end'] # .replace(['year_range'], ['year_start', 'year_end'])

### Merge and restore order of columns
clean = pd.concat([messy_int, messy_strings], axis=1) # on index level
clean = clean[column_order]


In [225]:
### Create dictionary with datatypes and update dataframe

# Partition columns into lists
string_cols = ['artist', 'tour_title']
date_cols = ['year_start', 'year_end']

convert_to_float = list(clean.columns)

for col in string_cols + date_cols:
    convert_to_float.remove(col) 

# create dictionary
data_type_colnames = convert_to_float + date_cols + string_cols
datatypes = ['float'] * len(convert_to_float) + ['int'] * len(date_cols) + ['str'] * len(string_cols)

datatype_conversion = dict(zip(data_type_colnames, datatypes))

# convert
clean = clean.astype(datatype_conversion)
clean


,rank,peak,all_time_peak,actual_gross,adjusted_gross_in_2022_dollars,artist,tour_title,year_start,year_end,shows,average_gross
0,1.0,1.0,2.0,780000000.0,780000000.0,Taylor Swift,The Eras Tour,2023,2024,56.0,13928571.0
1,2.0,1.0,7.0,579800000.0,579800000.0,Beyoncé,Renaissance World Tour,2023,2023,56.0,10353571.0
2,3.0,1.0,2.0,411000000.0,560622615.0,Madonna,Sticky Sweet Tour,2008,2009,85.0,4835294.0
3,4.0,2.0,10.0,397300000.0,454751555.0,Pink,Beautiful Trauma World Tour,2018,2019,156.0,2546795.0
4,5.0,2.0,NaN,345675146.0,402844849.0,Taylor Swift,Reputation Stadium Tour,2018,2018,53.0,6522173.0
5,6.0,2.0,10.0,305158363.0,388978496.0,Madonna,The MDNA Tour,2012,2012,88.0,3467709.0
6,7.0,2.0,NaN,280000000.0,381932682.0,Celine Dion,Taking Chances World Tour,2008,2009,131.0,2137405.0
7,8.0,NaN,NaN,257600000.0,257600000.0,Pink,Summer Carnival,2023,2024,41.0,6282927.0
8,9.0,NaN,NaN,256084556.0,312258401.0,Beyoncé,The Formation World Tour,2016,2016,49.0,5226215.0
9,10.0,NaN,NaN,250400000.0,309141878.0,Taylor Swift,The 1989 World Tour,2015,2015,85.0,2945882.0
